In [4]:
import os
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms, models
from torch import nn
from collections import OrderedDict
import matplotlib.pyplot as plt
import shutil

In [2]:
# mount colab to drive to access .zip file without uploading it everytime
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
!rm -r mel_spectrograms/

In [12]:
!unzip /content/gdrive/MyDrive/mel_spectrograms.zip

Streaming output truncated to the last 5000 lines.
  inflating: mel_spectrograms/train/yebapa1/XC440195.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC633712.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC438212.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC756765.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC464801.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC703822.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC289718.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC206029.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC516115.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC118442.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC268492.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC332548.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC265980.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC350293.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC332546.jpg  
  inflating: mel_spectrograms/train/yebapa1/XC356567.jpg  
  inf

In [13]:
# Define the paths to the train, test, and validation directories
train_dir = '/content/mel_spectrograms/train'
test_dir = '/content/mel_spectrograms/test'
validation_dir = '/content/mel_spectrograms/validation'

# Function to count files in subdirectories and return sorted list
def count_files_in_subdirs(directory):
    species_dirs = [os.path.join(directory, species) for species in os.listdir(directory) if os.path.isdir(os.path.join(directory, species))]
    species_file_counts = [(species, len(os.listdir(species))) for species in species_dirs]
    sorted_species = sorted(species_file_counts, key=lambda x: x[1], reverse=True)
    return sorted_species

# Function to delete non-top species directories
def delete_non_top_species_dirs(top_species_names, *directories):
    for directory in directories:
        for subdir in os.listdir(directory):
            subdir_path = os.path.join(directory, subdir)
            if subdir not in top_species_names and os.path.isdir(subdir_path):
                shutil.rmtree(subdir_path)  # Deletes the directory

# Count and sort the species in the train directory
sorted_species = count_files_in_subdirs(train_dir)

# Identify the top 20 species folder names
top_20_species_names = [os.path.basename(item[0]) for item in sorted_species[:20]]

# Print out the top 20 species for verification
print("Top 20 species based on the training samples:")
for species_name in top_20_species_names:
    print(species_name)

# Delete non-top-20 species directories from all folders
delete_non_top_species_dirs(top_20_species_names, train_dir, test_dir, validation_dir)

print("Non-top-20 species directories have been deleted from the train, test, and validation folders.")



Top 20 species based on the training samples:
barswa
wlwwar
comsan
eaywag1
thrnig1
woosan
combuz1
eubeat1
hoopoe
cohmar1
litegr
combul2
rbsrob1
blakit1
greegr
gnbcam2
rerswa1
somgre1
colsun2
ratcis1
Non-top-10 species directories have been deleted from the train, test, and validation folders.


In [14]:
import os
directory = "/content/mel_spectrograms/train"
lst1 = os.listdir(directory) # your directory path
number_files1 = len(lst1)
print(number_files1)

directory = "/content/mel_spectrograms/test"
lst2 = os.listdir(directory) # your directory path
number_files2 = len(lst2)
print(number_files2)

directory = "/content/mel_spectrograms/validation"
lst3 = os.listdir(directory) # your directory path
number_files3 = len(lst3)
print(number_files3)

21
21
21


In [15]:
output_folder = "mel_spectrograms"

# Define subfolders for training, validation, and testing
train_folder = os.path.join(output_folder, "train")
val_folder = os.path.join(output_folder, "validation")
test_folder = os.path.join(output_folder, "test")

In [16]:
directory = "/content/mel_spectrograms/train"
lst3 = os.listdir(directory) # your directory path
number_files3 = len(lst3)
print(number_files3)

21


In [17]:
data_transforms = transforms.Compose([transforms.RandomResizedCrop(224), # before all 225 ##256
                                      #transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])

# Load the datasets with ImageFolder
train_data = ImageFolder(train_folder, data_transforms)
valid_set = ImageFolder(val_folder, data_transforms)
test_set = ImageFolder(test_folder, data_transforms)

# Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size =64, shuffle=True)
valid_data = torch.utils.data.DataLoader(valid_set, batch_size=64)
test_data = torch.utils.data.DataLoader(test_set, batch_size=64)

classes = dict(zip(list(range(len(train_data.classes))),train_data.classes))

In [18]:
model = models.vit_b_16(weights='DEFAULT')
model

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 138MB/s]


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [19]:
#freeze parameters
for param in model.parameters():
    param.requires_grad = False

#new classifier for the feedforward
classifier = nn.Sequential(OrderedDict([
                            ('fc1', nn.Linear(768, 400)), #4608,320
                            ('relu', nn.ReLU()),
                            ('dropout', nn.Dropout(0.5)),
                            ('fc2', nn.Linear(400, 350)), #320,300
                            ('relu', nn.ReLU()),
                            ('dropout', nn.Dropout(0.5)),
                            ('fc3', nn.Linear(350, 247)), #300,102
                            ('output', nn.LogSoftmax(dim=1))
                            ]))
model.heads = classifier
print(model)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [20]:
def create_backup(filename, train_losses, valid_losses, valid_acc):
    torch.save(train_losses, "/content/gdrive/MyDrive/bird_backups/train_losses_"+filename)
    torch.save(valid_losses, "/content/gdrive/MyDrive/bird_backups/valid_losses_"+filename)
    torch.save(valid_acc, "/content/gdrive/MyDrive/bird_backups/valid_acc_"+filename)
    torch.save(model,'/content/gdrive/MyDrive/bird_backups/model_'+filename+'.pth')
    # Print loss
    cpu_train_losses = []
    cpu_valid_losses = []
    cpu_valid_acc = []

    for tens in train_losses:
        cpu_train_losses.append(tens.to('cpu'))
    plt.plot(cpu_train_losses, label='Training loss')
    plt.plot(valid_losses, label='Validation loss')
    plt.legend(frameon=False)
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.savefig('/content/gdrive/MyDrive/bird_backups/fig/plot_loss_'+filename+'.jpg')

    plt.clf()
    plt.plot(valid_acc, label='Validation Accuracy')
    plt.legend(frameon=False)
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.savefig('/content/gdrive/MyDrive/bird_backups/fig/plot_acc_'+filename+'.jpg')

    print("Created tensor backup")

In [21]:
def validation(model, valid_data, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in valid_data:

        images,labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images)
        batch_loss = criterion(output, labels)
        test_loss += batch_loss.item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()

    return test_loss, accuracy

In [22]:
train_losses, valid_losses, valid_acc = [], [], []


In [ ]:
learning_rate=0.000005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.functional.nll_loss
epochs = 500
offset=0
step_of_backup=10
model.to('cuda')
for epoch in range(epochs):
  model.train()
  counter = 0
  for data in trainloader:
    X , y= data
    X , y = X.to('cuda'), y.to('cuda')
    optimizer.zero_grad() # clear gradient information.
    #output = model(X) <- gives errors
    output = model.forward(X)
    loss = criterion(output, y)
    loss.backward() # do pack-propagation step
    optimizer.step() # tell optimizer that you finished batch/iteration.
    counter += 1

  # network in eval mode for inference
  model.eval()

  # Turn off gradients for validation, saves memory and computations
  with torch.no_grad():
      test_loss, accuracy = validation(model, valid_data, criterion)
  print("Epoch: {}/{}...".format(epoch+1,epochs),
        "Loss: {:.4f}".format(loss.data),
        "Train Loss: {:.3f}.. ".format(test_loss/len(valid_data)),
        "Train Accuracy: {:.3f}".format(accuracy/len(valid_data)))
  train_losses.append(loss.data)
  valid_losses.append(test_loss/len(valid_data))
  valid_acc.append(accuracy/len(valid_data))
  running_loss = 0
  model.train()
  if((epoch+1) % step_of_backup == 0):
    create_backup('checkpoint_epoch_'+str(epoch+1+offset)+'_lr_'+str(learning_rate)+'.txt', train_losses, valid_losses, valid_acc)

Epoch: 1/500... Loss: 5.4458 Train Loss: 5.445..  Train Accuracy: 0.016
Epoch: 2/500... Loss: 5.3694 Train Loss: 5.345..  Train Accuracy: 0.058
Epoch: 3/500... Loss: 5.1959 Train Loss: 5.216..  Train Accuracy: 0.069
Epoch: 4/500... Loss: 5.0907 Train Loss: 5.031..  Train Accuracy: 0.076
Epoch: 5/500... Loss: 4.8470 Train Loss: 4.787..  Train Accuracy: 0.079
Epoch: 6/500... Loss: 4.5387 Train Loss: 4.471..  Train Accuracy: 0.072
Epoch: 7/500... Loss: 4.1563 Train Loss: 4.103..  Train Accuracy: 0.070
Epoch: 8/500... Loss: 3.6653 Train Loss: 3.755..  Train Accuracy: 0.075
Epoch: 9/500... Loss: 3.4999 Train Loss: 3.484..  Train Accuracy: 0.082
Epoch: 10/500... Loss: 3.3554 Train Loss: 3.298..  Train Accuracy: 0.093
Created tensor backup
Epoch: 11/500... Loss: 3.1117 Train Loss: 3.190..  Train Accuracy: 0.108
Epoch: 12/500... Loss: 3.1888 Train Loss: 3.104..  Train Accuracy: 0.129
Epoch: 13/500... Loss: 3.1768 Train Loss: 3.040..  Train Accuracy: 0.131
Epoch: 14/500... Loss: 3.0522 Train Lo